In [132]:
class Maze:
    '''
    This is the main class to create maze.
    '''
    def __init__(self,agent,rows=4,cols=4):
        '''
        rows--> No. of rows of the maze
        cols--> No. of columns of the maze
        Need to pass just the two arguments. The rest will be assigned automatically
        maze_map--> Will be set to a Dicationary. Keys will be cells and
                    values will be another dictionary with keys=['E','W','N','S'] for
                    East West North South and values will be 0 or 1. 0 means that 
                    direction(EWNS) is blocked. 1 means that direction is open.
        grid--> A list of all cells
        path--> Shortest path from start(bottom right) to goal(by default top left)
                It will be a dictionary
        _win,_cell_width,_canvas -->    _win and )canvas are for Tkinter window and canvas
                                        _cell_width is cell width calculated automatically
        _agents-->  A list of aganets on the maze
        markedCells-->  Will be used to mark some particular cell during
                        path trace by the agent.
        _
        '''
        self.rows=rows
        self.cols=cols
        self.ix = agent.ix ## pos of agent on rows 
        self.iy = agent.iy ## pos of agent on cols
        self.visited_beg = []
        ## Saving all our visited starting point
        if (self.ix,self.iy) not in self.visited_beg :
            self.visited_beg.append((self.ix,self.iy)) 
            
        self.eps = agent.eps ## eps determinated in agent class
        
        self.start= None
        self.end=None
        self.isFeasable = False ## If there exist a path between start and end point set the false because there is no keypoint at the beggening 
        
        ### path between Start - End point
        self.path=[]
        ### set dist between Start and End to None because there is no path at the beggening
        self.dist_SE=None
        self.reward=0
        
        self.actions = ["addStart","addEnd","openEast","openWest","openNorth","openSouth","goRight","goLeft","goUp",
        "goLeft"]
        ## Binary number set to True the first time we reach dist Start and end =3
        self.reached_dist = False
        ## At the beggening there is 0 connection
        self.nbConnection = 0
        self.maze_map = {}
        for x in range(self.rows):
            for y in range(self.cols):
                self.maze_map[x,y]={'E':0,'W':0,'N':0,'S':0}
                
        self.len_actions = len(self.actions)
        ### first initiale state with all the walls closed
        self.state = hash(str(self.maze_map)+str(self.start)+str(self.end)+str((self.ix,self.iy)))
        ### add our first state to our Q_hash
        self.Q_hash = {self.state:[0]*self.len_actions}
        ### keep visited_state at each game dict : {key = state : value = action(state)}
        self.visited_state = {self.state:0} 
        
        
    def __str__(self):
        """Return a (crude) string representation of the maze."""

        maze_rows = ['-' * self.rows * 2]
        for x in range(self.rows):
            maze_row = ['|']
            for y in range(self.cols):
                if x == 0 and y == 0:
                    maze_row.append('S')
                elif x == 3 and y == 3:
                    maze_row.append('E')
                elif x == 1 and y == 2:
                    maze_row.append('T')
                if not self.maze_map[x,y]['E']:
                    maze_row.append(' |')
                else:
                    maze_row.append('  ')
            maze_rows.append(''.join(maze_row))
            maze_row = ['|']
            for y in range(self.cols):
                if not self.maze_map[x,y]['N']:
                    maze_row.append('-+')
                else:
                    maze_row.append(' +')
            maze_rows.append(''.join(maze_row))
        
        return '\n'.join(maze_rows)
    def write_svg(self, filename):
        """Write an SVG image of the maze to filename."""

        aspect_ratio = self.rows / self.cols
        # Pad the maze all around by this amount.
        padding = 10
        # Height and width of the maze image (excluding padding), in pixels
        height = 500
        width = int(height * aspect_ratio)
        # Scaling factors mapping maze coordinates to image coordinates
        scy, scx = height / self.cols, width / self.rows

        def write_wall(ww_f, ww_x1, ww_y1, ww_x2, ww_y2):
            """Write a single wall to the SVG image file handle f."""

            print('<line x1="{}" y1="{}" x2="{}" y2="{}"/>'
                  .format(ww_x1, ww_y1, ww_x2, ww_y2), file=ww_f)

        # Write the SVG image file for maze
        with open(filename, 'w') as f:
            # SVG preamble and styles.
            print('<?xml version="1.0" encoding="utf-8"?>', file=f)
            print('<svg xmlns="http://www.w3.org/2000/svg"', file=f)
            print('    xmlns:xlink="http://www.w3.org/1999/xlink"', file=f)
            print('    width="{:d}" height="{:d}" viewBox="{} {} {} {}">'
                  .format(width + 2 * padding, height + 2 * padding,
                          -padding, -padding, width + 2 * padding, height + 2 * padding),
                  file=f)
            print('<defs>\n<style type="text/css"><![CDATA[', file=f)
            print('line {', file=f)
            print('    stroke: #000000;\n    stroke-linecap: square;', file=f)
            print('    stroke-width: 5;\n}', file=f)
            print(']]></style>\n</defs>', file=f)
            # Draw the "South" and "East" walls of each cell, if present (these
            # are the "North" and "West" walls of a neighbouring cell in
            # general, of course).
            for x in range(self.rows):
                for y in range(self.cols):
                    if not self.maze_map[y, x]['N']:
                        x1, y1, x2, y2 = x * scx, (y + 1) * scy, (x + 1) * scx, (y + 1) * scy
                        write_wall(f, x1, y1, x2, y2)
                    if not self.maze_map[y, x]['E']:
                        x1, y1, x2, y2 = (x + 1) * scx, y * scy, (x + 1) * scx, (y + 1) * scy
                        write_wall(f, x1, y1, x2, y2)
            # Draw the North and West maze border, which won't have been drawn
            # by the procedure above.
            print('<line x1="0" y1="0" x2="{}" y2="0"/>'.format(width), file=f)
            print('<line x1="0" y1="0" x2="0" y2="{}"/>'.format(height), file=f)
            print('</svg>', file=f)
            

    ### reset the env
    def reset(self):
        for x in range(self.rows):
            for y in range(self.rows):
                self.maze_map[x,y]={'E':0,'W':0,'N':0,'S':0}
        self.start = None
        self.end = None
        self.ix , self.iy = np.random.randint(self.rows),np.random.randint(self.cols)
        if (self.ix,self.iy) not in self.visited_beg :
            self.visited_beg.append((self.ix,self.iy))
        self.dist_SE = None
        self.reward = 0
        # self.treasure = None 
        self.state = hash(str(self.maze_map)+str(self.start)+str(self.end)+str((self.ix,self.iy)))
        ###Q_hash doesn't reset thus it can be possible that this state was already visited
        if not self.state in self.Q_hash.keys():
            self.Q_hash[self.state] = [0]*self.len_actions
        self.visited_state = {self.state:0}
        

            
        
    def take_actions(self,eps):
        """
        randomly chose an action with proba eps otherwise take the best action given state : self.state
        """
        if np.random.random() < eps : 
            return np.random.randint(self.len_actions)
        else : 
            return np.argmax(self.Q_hash[self.state])
            
    def update_states(self,action_index):
        """
        Update state with respect to action_index then get the state from :str(self.maze_map)+str(self.start)+str(self.end), and stock his hash 
        self.state hash(str(self.maze_map)+str(self.start)+str(self.end))
        if it's a new state we add it on our Q_hash and then we initialize self.Q_hash [self.state] = [0]*number of possible actions 
        and we add self.state in our visited_state dictionary 
        """
        if self.actions[action_index] == "openEast" :
            self._Open_East()
            
        elif self.actions[action_index] == "openWest" :
            self._Open_West()
            
        elif self.actions[action_index] == "openNorth" :
            self._Open_North()
            
        elif self.actions[action_index] == "openSouth" :
            self._Open_South()
            
        elif self.actions[action_index] == "goRight" :
            self._Right()
            
        elif self.actions[action_index] == "goLeft" :
            self._Left()
            
        elif self.actions[action_index] == "goUp" :
            self._Up()
            
        elif self.actions[action_index] == "goDown" :
            self._Down()
            
        elif self.actions[action_index] == "addStart" :
            self._Add_Start()
            
        elif self.actions[action_index] == "addEnd" :
            self._Add_End()
        # print(self.actions[action_index])
            
        self.state = hash(str(self.maze_map)+str(self.start)+str(self.end)+str((self.ix,self.iy)))
        ### If it's a new state add it on our Q_hash
        if not self.state in self.Q_hash.keys():
            self.Q_hash[self.state] = [0]*self.len_actions
        self.visited_state[self.state] = action_index
        
        self.update_path()
        
        
    def update_path(self):
        ###  check at each step if the maze become feasable and set isFeasable to True if so
        self.path = self.BFS(self.start,self.end)
        if self.end in self.path :
            self.isFeasable = True
            self.dist_SE = len(self.path) - 1
            
        ### otherwhise set isFeasable to false
        if not self.end in self.path :
            self.isFeasable = False 
        
        
    ## agent move to bottom cell if it's not a edge  
    def _Down(self):
        if self.maze_map[self.ix,self.iy]['S'] == True :
            self.ix = self.ix-1  
            
            
            
    def _Up(self):
        if self.maze_map[self.ix,self.iy]['N'] == True :
            self.ix = self.ix+1  
            
            
            
    def _Left(self):
        if self.maze_map[self.ix,self.iy]['W'] == True :
            self.iy = self.iy-1  
            
            
            
    def _Right(self):
        if self.maze_map[self.ix,self.iy]['E'] == True :
            self.iy = self.iy+1 
    
    def _Add_End(self):
        ### if there is already a key point do nothing 
        # print("laEnd")
        if self.start != (self.ix,self.iy) :
            self.end = (self.ix, self.iy)
        # print("iciEnd")
    
    def _Add_Start(self):
        # print("laStart")
        if self.end != (self.ix,self.iy) :
            self.start = (self.ix, self.iy)
        # print("iciStart")

    ### Open east wall if it's close, close it if it's open                              
    def _Open_East(self):
        '''
        To change the East Wall of the cell
        '''
        ### Open if it's close 
        if self.maze_map[self.ix,self.iy]['E']==0:
            if self.iy+1<self.cols:
                self.maze_map[self.ix,self.iy]['E']=1
                self.maze_map[self.ix,self.iy+1]['W']=1
                self.nbConnection += 1
        ### Close if it's open     
        else :
            if self.iy+1<self.cols:
                self.maze_map[self.ix,self.iy]['E']=0
                self.maze_map[self.ix ,self.iy+1]['W']=0
                self.nbConnection -= 1
            
    def _Open_West(self):
        if self.maze_map[self.ix,self.iy]['W']==0 :
            if self.iy-1>=0:
                self.maze_map[self.ix,self.iy]['W']=1
                self.maze_map[self.ix,self.iy-1]['E']=1
                self.nbConnection += 1   
        else :
            if self.iy-1>=0:
                self.maze_map[self.ix,self.iy]['W']=0
                self.maze_map[self.ix,self.iy-1]['E']=0
                self.nbConnection -= 1
            
            
            
    def _Open_North(self):
        if self.maze_map[self.ix,self.iy]['N']==0:
            if self.ix+1<self.rows:
                self.maze_map[self.ix,self.iy]['N']=1
                self.maze_map[self.ix+1,self.iy]['S']=1
                self.nbConnection += 1
        else :
            if self.ix+1<self.rows:
                self.maze_map[self.ix,self.iy]['N']=0
                self.maze_map[self.ix+1,self.iy]['S']=0
                self.nbConnection -= 1
            
            
    def _Open_South(self):
        if self.maze_map[self.ix,self.iy]['S']==0:
            if self.ix-1>=0:
                self.maze_map[self.ix,self.iy]['S']=1
                self.maze_map[self.ix-1,self.iy]['N']=1
                self.nbConnection += 1
        else : 
            if self.ix-1>=0:
                self.maze_map[self.ix,self.iy]['S']=0
                self.maze_map[self.ix-1,self.iy]['N']=0
                self.nbConnection -= 1
               
                    
    ### to find path between start and end point
    def BFS(self,from_,to_):
        ## Do BFS only there is a start and
        start = from_
        end = to_ 
        path = {}
        if from_ and to_ :
            frontier = [start]
            visited =[start]
            while len(frontier)>0 :
                currCell = frontier.pop(0) #first in first out
                for d in 'ESNW':
                    if self.maze_map[currCell][d] == True :
                        if d=="E":
                            childCell=(currCell[0],currCell[1]+1)
                        elif d=="S":
                            childCell=(currCell[0]-1,currCell[1])
                        elif d=="N":
                            childCell=(currCell[0]+1,currCell[1])
                        elif d=="W":
                            childCell=(currCell[0],currCell[1]-1) 
                        if childCell in visited:
                            continue
                        frontier.append(childCell)
                        visited.append(childCell)
                        path[childCell]=currCell
                        if currCell == end :
                            break
        ## keeping only the working path 
        if not end in path.keys() :
            return []
        fwdPath = {}
        
        cell = end
        while cell != start :
            fwdPath[path[cell]] = cell
            cell = path[cell]
        return [end] + list(fwdPath.keys())

           
    def open_try(self):
        for x in range(maze.rows):
            for y in range(maze.cols):
                random = np.random.randint(4)
                self.ix,self.iy = (x,y)
                if random ==0:
                    self._Open_East()
                elif random ==1 :
                    self._Open_West()
                elif random == 2:
                    self._Open_South()
                elif random == 3:
                   self._Open_North() 
                   
    def give_reward(self,prev_isFeasable,prev_distSE,prev_nbConnection,prev_start,prev_end):
        reward = 0
        ## the first time we add a starting point
        if not prev_start == True and self.start :
            reward +=4
        ## the first time we add a starting point
        if not prev_end == True and self.end :
            reward +=4
            
        if self.isFeasable == True :
            if self.dist_SE >=5 and self.reached_dist == False :
                    reward += 20
                    self.reached_dist = True
                    
            if prev_isFeasable == True :
                reward += self.dist_SE - prev_distSE
            elif prev_isFeasable == False :
                reward += 8
                
        if prev_isFeasable == True and self.isFeasable == False :
            reward -= 8
        
        if prev_distSE >=3 and self.isFeasable == False:
            reward -= 4
        
        if prev_nbConnection > self.nbConnection :
            reward -= 1
        elif prev_nbConnection < self.nbConnection :
            reward += 1
            
        return reward


In [120]:
if False and (2,3) :
    print('a')

In [133]:

import numpy as np
class Agent():
    """
    alpha : learning rate 
    gamma : discount factor 
    eps : exploration/exploitation greedy score
    """
    def __init__(self,name="first_game", alpha=0.2, gamma=0.9, eps=0.2,rand_range=4):
        self.name = name
        self.eps= eps
        self.gamma = gamma
        self.alpha = alpha
        self.ix = np.random.randint(rand_range)
        self.iy = np.random.randint(rand_range)
        self.reward = 0
    
    def reset_agent(self):
        self.ix = np.random.randint(rand_range)
        self.iy = np.random.randint(rand_range)
        self.reward = 0
        return(self.ix,self.iy)
    
    

In [134]:
agent = Agent()
maze = Maze(agent)
j=0
for epochs in range(10000):
    for step in range(100):
        ## choose best action with respect to current Q table 
        isFeasable = maze.isFeasable
        ## current_ncConnection
        current_nbConnection = maze.nbConnection
        ## current state 
        current_state = maze.state
        current_action_index = maze.take_actions(agent.eps)
        current_q_value = maze.Q_hash[current_state][current_action_index]
        current_start = maze.start 
        current_end = maze.end 
        ## reset the current reward
        ## update state with respect to  the current best action 
        maze.update_states(current_action_index)
        maze.update_path()
        
        ## the reward function is highly depending of the prev_state
        reward = maze.give_reward(current_state,isFeasable,current_nbConnection,current_start,current_end)
        
        ## new best action with respect to new Q table, we don't want to explore here so eps = 0
        new_action_index = maze.take_actions(0)
        new_state = maze.state 
        new_q_value = maze.Q_hash[new_state][new_action_index]
        ##bellman equation 
        temporal_difference = reward + agent.gamma * new_q_value - current_q_value
        
        maze.Q_hash[current_state][current_action_index] = current_q_value + (agent.alpha * temporal_difference)
    maze.reset()
    
   


In [ ]:
maze.reset()
# maze.ix , maze.iy= 1,1
print(f'x : {maze.ix} y : {maze.iy}')
print(f'start : {maze.start} end : {maze.end}')
print("-"*40)

for step in range(100):
        ## choose best action with respect to current Q table
        current_action_index = maze.take_actions(0)
        ## update state with respect to  the current best action 
        maze.update_states(current_action_index)
        
# print(maze.__str__()) 
print(f'start : {maze.start} end : {maze.end}')
maze.write_svg("test.svg")  

x : 0 y : 3
start : None end : None
----------------------------------------
start : (1, 1) end : (1, 2)


In [105]:
maze.BFS(maze.start,maze.end)
# agent = Agent()
# maze = Maze(agent)
# maze.open_try()
# print(maze.ix,maze.iy)
# print(maze.__str__())
# maze.write_svg("test.svg")
# print(maze.maze_map[maze.ix,maze.iy])

[]

In [13]:
maze.BFS(maze.start,maze.end)

[]

In [14]:
maze.start,maze.end

(None, None)

In [32]:
maze.reset()
# maze.ix , maze.iy= 1,1
print(f'x : {maze.ix} y : {maze.iy}')
print(f'start : {maze.start} end : {maze.end}')
print("-"*40)

for step in range(100):
        ## choose best action with respect to current Q table
        # print(maze.actions[current_action_index])
        print(f"loc {(maze.ix,maze.iy)}")
        print(maze.Q_hash[maze.state])
        current_action_index = maze.take_actions(0)
        ## update state with respect to  the current best action 
        maze.update_states(current_action_index)
        
# print(maze.__str__()) 
print(f'start : {maze.start} end : {maze.end}')
maze.write_svg("test.svg")  

x : 0 y : 1
start : None end : None
----------------------------------------
loc (0, 1)
[16.199999913680113, 0.4924944, 3.095956419608943, 0.0, 16.199999999592315, 16.199891049047544, 30.254355373522962, 26.567990637388018, 49.99999999999984, 3.13868656]
laStart
iciStart
addStart
loc (0, 1)
[44.99999999999984, 40.49999999999984, 44.99999999999984, 44.99999999999984, 44.99999999999984, 44.99999999999984, 44.9999999999998, 44.99999999999984, 44.99999999999955, 49.99999999999984]
laEnd
iciEnd
addEnd
loc (0, 1)
[44.99999999999984, 40.49999999999984, 44.99999999999984, 44.99999999999984, 44.99999999999984, 44.99999999999984, 44.9999999999998, 44.99999999999984, 44.99999999999955, 49.99999999999984]
laEnd
iciEnd
addEnd
loc (0, 1)
[44.99999999999984, 40.49999999999984, 44.99999999999984, 44.99999999999984, 44.99999999999984, 44.99999999999984, 44.9999999999998, 44.99999999999984, 44.99999999999955, 49.99999999999984]
laEnd
iciEnd
addEnd
loc (0, 1)
[44.99999999999984, 40.49999999999984, 44.999

In [6]:
print(maze.state)
print(hash(str(maze.maze_map)+str(maze.start)+str(maze.end)+str((maze.ix,maze.iy))))

In [7]:
Q=pd.DataFrame(maze.Q_hash).T
Q.columns = maze.actions
Q